In [ ]:
import pandas as pd
import os

import config

from utils.evaluation_utils import print_ensemble_results, get_best_epoch

In [ ]:
# PATH_TO_ANOMSCORES = config.PATH_TO_ANOM
# PATH_TO_RESULTS = config.PATH_TO_RESULTS
# path_to_scores = os.path.join(PATH_TO_ANOMSCORES, 'ss')
# path_to_results = os.path.join(PATH_TO_RESULTS, 'ss')
# print_ensemble_results(path_to_anom_scores=path_to_scores, epoch=400, stage='ss', metric='centre_mean', meta_data_dir=None, get_oarsi_results=False, model_name_prefix="mod_1", seed = 1001)
# metrics = ['auc', 'auc_mid', 'auc_mid2', 'auc_sev', 'spearman']
# best_epochs = get_best_epoch(path_to_results, last_epoch=0, metric='auc', model_prefix='mod_1')
# best_epochs

## Visualize Results AS

### KL Score vs. Anom Count

In [ ]:
from utils.label_eval_utils import *

In [ ]:
x = 'label'
y = 'anoms_count'
values = ['label', 'anoms_count', 'sim', 'av']
x_title = 'KL Score'
y_title = 'Count Anomalities'

SAVE_PATH = config.OUTPUT_PATH
pseudolabels_path= os.path.join(SAVE_PATH, 'pseudolabels')

pseudolabel_name = "ss_training_mod_2_epoch_400_margin_0.8.csv"

# save_path = os.path.join(SAVE_PATH, 'graphs')
# os.makedirs(save_path, exist_ok=True)

In [ ]:
df = pd.read_csv(os.path.join(pseudolabels_path, pseudolabel_name), index_col=False)

In [ ]:
df['anoms_count'].value_counts()

In [ ]:
df.columns

In [ ]:
scatter_plot(df, x=x, y=y, x_title=x_title,y_title=y_title)

In [ ]:
boxplot(df, x, x_title, y, y_title)

In [ ]:
df.head()

In [ ]:
x = 'label'
y = 'av'
x_title = 'KL Score'
y_title = 'Average AS'

In [ ]:
scatter_plot(df, x=x, y=y, x_title=x_title,y_title=y_title)

In [ ]:
def boxplot(df, x = 'label', x_title = 'KL Score', y = 'anoms_count', y_title = 'Count Anomalities', title = None, save_path = None):
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=x, y=y, data=df)
    if title is not None:
        plt.title('Box Plot: Distribution of Anoms Count by Label')
    plt.xlabel(x_title)
    plt.ylabel(y_title)
    plt.show()
    if save_path is not None:
        plt.savefig(save_path)

In [ ]:
l = ['col_1001', 'col_71530', 'col_138647',
       'col_875688', 'col_985772', 'col_44', 'col_34', 'col_193', 'col_244959',
       'col_8765']

x = 'label'
x_title = 'KL Score'

#if I have a boxplot for every "seed" is there a better way of visualizing the variability that we see for the same samples across seeds?
# specifically also for the cases where we have inconsistent anomalie "labels"

for i in l:
    boxplot(df, x = x)

In [ ]:
x = 'sim'
y = 'av'
x_title = 'CLIP Sim Score'
y_title = 'Average AS'

In [ ]:
def scatter_plot_wlegend(df, x = 'label', x_title = 'KL Score', y = 'anoms_count', y_title = 'Count Anomalities', label = None, title = None, save_path = None): # y = 'anom_av', y_title = 'Average Anom Score'
    '''
    Get a scatter plot between the KL score and anom_count (so how often the sample is considered to be an anomality) OR the anom score
    '''
    plt.figure(figsize=(16, 12))
    if label is not None:
        plt.scatter(df[x], df[y], alpha=0.6, c = df[label], cmap='viridis')
        plt.colorbar(label=label)
    else:
        plt.scatter(df[x], df[y], alpha=0.6, color='blue')
    if title is not None:
       plt.title(title)
    plt.xlabel(x_title)
    plt.ylabel(y_title)
    plt.grid(True)
    plt.show()
    if save_path is not None:
        plt.savefig(os.path.join(save_path, "test_image.png"))

In [ ]:
scatter_plot_wlegend(df[(df['anoms_count']<=5) & (df['anoms_count']>0)], x=x, y=y, x_title=x_title,y_title=y_title, label='label', title="OAI - AS Score vs CLIP Sim Score (m=0.8)")

#### Pairplots

In [ ]:
pairplot(df, values)

In [ ]:
sns.pairplot(df[values], hue='label', palette='viridis')
plt.show()

In [ ]:
# g = sns.PairGrid(df[values], hue='label')
# g.map_lower(sns.boxplot) 
# g.map_diag(sns.histplot)  # Apply histogram for the diagonal
# g.add_legend()
# plt.show()

# Example X-Rays

## High CLIP Similarity Score for Different KL scores

In [ ]:
ids = df[df['sim>95th']>0]

file_name = pseudolabel_name.rsplit('.', maxsplit=1)[0] + '_' + 'simscore_over_95th.csv'
save_path_ids = os.path.join(SAVE_PATH, 'eval', 'ss')
os.makedirs(save_path_ids, exist_ok=True)

ids.to_csv(os.path.join(save_path_ids, file_name), index=False)